In [1]:
import glob
import pandas as pd
# import tensorflow as tf
import os
from typing import Generator
from datetime import datetime
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [2]:
def txt_file_generator(file_list: list) -> Generator[pd.DataFrame, None, None]:
    for txt_file in file_list:
        yield read_txt_file(txt_file)

In [3]:
def find_txt_files(directory):
    txt_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):    
#                 print(os.path.join(root, file))
                txt_files.append(os.path.join(root, file))
    return txt_files

In [4]:
def read_txt_file(file_path):
    # 从文件名中提取年、月、日
    date_str = os.path.basename(file_path).split('.')[0] # 例如，得到"20100101"
    year = int(date_str[:4])
    month = int(date_str[4:6])
    day = int(date_str[6:])

    df = pd.read_csv(file_path, delim_whitespace=True, header=None)
    
    # 将年、月、日添加为新列
    df['year'] = year
    df['month'] = month
    df['day'] = day

    return df


火险指数
---

In [5]:
txt_files = find_txt_files("/public/home/lihf_hx/yyc/建模数据/model_CHINA")

dataframes = []  
for txt_file in txt_files:
    df = read_txt_file(txt_file)
    dataframes.append(df)
df_CHINA = pd.concat(dataframes)  
df_CHINA.columns =['sta','lon','lat','year','month','day','FFDI']
df_CHINA.to_csv('/public/home/lihf_hx/yyc/森林火险模型/CHINA.csv')


KeyboardInterrupt



In [ ]:
df_CHINA.head()

美国模型
---

In [11]:
txt_files = find_txt_files("/public/home/lihf_hx/yyc/建模数据/model_USA")

df_gen = txt_file_generator(txt_files)
df_USA = pd.concat(df_gen, axis=0)

df_USA.columns =['sta','lon','lat','yth1','yth10','yth100','yth1000','kb','erc','sc','bi','ic','p','year','month','day']    
df_USA.to_csv('/public/home/lihf_hx/yyc/森林火险模型/dataset/USA.csv')

In [12]:
df_USA.head()

,sta,lon,lat,yth1,yth10,yth100,yth1000,kb,erc,sc,bi,ic,p,year,month,day
0,50136,122.31,52.58,12.53,15.06,16.14,15.69,9.09,0.0,0.0,0.0,0.0,0.28,2010,1,9
1,50246,124.43,52.21,11.89,14.24,15.02,14.42,65.33,0.0,0.0,0.0,0.0,0.39,2010,1,9
2,50247,123.34,52.02,12.15,14.86,17.28,16.18,26.28,0.0,0.0,0.0,0.0,0.31,2010,1,9
3,50349,124.20,51.42,10.76,13.31,16.23,15.50,20.99,0.0,0.0,0.0,0.0,0.30,2010,1,9
4,50353,126.39,51.43,10.38,12.60,14.17,13.98,91.91,0.0,0.0,0.0,0.0,0.45,2010,1,9


加拿大
---

In [13]:
txt_files = find_txt_files("/public/home/lihf_hx/yyc/建模数据/model_CANADA")

df_gen = txt_file_generator(txt_files)
df_CANADA = pd.concat(df_gen, axis=0)

df_CANADA.columns =['sta','lon','lat','FFMC','DMC','DC','FWI','ISI','BUI','DSR','year','month','day']
df_CANADA.to_csv('/public/home/lihf_hx/yyc/森林火险模型/dataset/CANADA.csv')

匹配
---

In [15]:
df_USA = pd.read_csv('/public/home/lihf_hx/yyc/森林火险模型/dataset/USA.csv', index_col=0)
df_CANADA = pd.read_csv('/public/home/lihf_hx/yyc/森林火险模型/dataset/CANADA.csv', index_col=0)
df_CHINA = pd.read_csv('/public/home/lihf_hx/yyc/森林火险模型/dataset/CHINA.csv', index_col=0)

# df_USA = df_USA.drop_duplicates(subset= ['sta'],keep='first',inplace=False)
# df_CANADA = df_CANADA.drop_duplicates(subset= ['sta'],keep='first',inplace=False)
merged_uc = df_USA.merge(df_CANADA, on=['sta', 'lon', 'lat','year','month','day'])

In [16]:
merged_uc = merged_uc.drop(columns=['lon', 'lat'])
merged_all =merged_uc.merge(df_CHINA, on = ['sta','year','month','day'])
# merged_all = merged_all.drop_duplicates(subset= ['sta'],keep='first',inplace=False)

In [17]:
merged_all

,sta,yth1,yth10,yth100,yth1000,kb,erc,sc,bi,ic,...,FFMC,DMC,DC,FWI,ISI,BUI,DSR,lon,lat,FFDI
0,50136,12.53,15.06,16.14,15.69,9.09,0.00,0.00,0.00,0.0,...,52.39,2.28,122.06,0.10,0.24,4.36,0.00,122.52,52.97,0.00
1,50246,11.89,14.24,15.02,14.42,65.33,0.00,0.00,0.00,0.0,...,73.77,13.86,141.33,2.05,1.38,22.26,0.10,124.72,52.35,0.00
2,50247,12.15,14.86,17.28,16.18,26.28,0.00,0.00,0.00,0.0,...,55.81,5.31,80.40,0.21,0.37,9.11,0.00,123.57,52.03,0.00
3,50349,10.76,13.31,16.23,15.50,20.99,0.00,0.00,0.00,0.0,...,69.17,6.13,93.80,0.64,1.03,10.54,0.01,124.33,51.70,0.00
4,50353,10.38,12.60,14.17,13.98,91.91,0.00,0.00,0.00,0.0,...,81.50,15.32,210.45,4.35,2.27,25.92,0.37,126.65,51.72,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6441926,59941,9.91,12.50,16.49,22.66,198.10,4.99,0.18,2.83,3.0,...,89.14,23.50,62.22,10.39,6.07,24.18,1.71,109.52,18.77,0.41
6441927,59945,9.47,11.89,15.39,20.44,243.68,5.55,0.30,3.79,5.0,...,89.95,28.27,77.42,14.85,8.33,29.56,3.23,109.70,18.65,0.44
6441928,59948,10.92,13.38,15.65,21.80,186.86,11.64,4.70,18.97,5.0,...,87.21,13.70,72.68,16.33,12.72,18.63,3.82,109.59,18.22,0.48
6441929,59951,10.46,12.74,14.54,23.66,148.13,13.07,2.07,13.70,3.0,...,88.61,21.19,58.93,12.94,8.39,22.32,2.53,110.33,18.80,0.33


In [18]:
merged_all.to_csv('/public/home/lihf_hx/yyc/森林火险模型/dataset/model.csv')

In [19]:
merged_all.head(100).to_csv('/public/home/lihf_hx/yyc/森林火险模型/dataset/model100.csv')
